In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import tensorflow as tf
import time

C:\Users\Michal\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [29]:
def tf_rad2deg(rad):
    pi_on_180 = 0.017453292519943295
    return rad / pi_on_180

def dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

# takes 1 dimensional tensor and outputs an angle
def dihedral_tf1(p):
    p0 = tf.gather(p, 0)
    p1 = tf.gather(p, 1)
    p2 = tf.gather(p, 2)
    p3 = tf.gather(p, 3)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1))
    
    v = tf.subtract(b0, tf.multiply(tf.tensordot(b0, b1, 1), b1))
    w = tf.subtract(b2, tf.multiply(tf.tensordot(b2, b1, 1), b1))
    
    x = tf.tensordot(v, w, 1)
    y = tf.tensordot(tf.cross(b1, v), w, 1)
    
    return tf_rad2deg(tf.atan2(y,x))

# takes 2 dimensional tensor (K, 4) and outputs K angles
def dihedral_tf2(p):
    p0 = tf.gather(p, 0, axis=1)
    p1 = tf.gather(p, 1, axis=1)
    p2 = tf.gather(p, 2, axis=1)
    p3 = tf.gather(p, 3, axis=1)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1, axis=1, keepdims=True))
    
    v = tf.subtract(b0, tf.einsum('b,bi->bi', tf.einsum('bi,bi->b', b0, b1), b1))
    w = tf.subtract(b2, tf.einsum('b,bi->bi', tf.einsum('bi,bi->b', b2, b1), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 1, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 1, keepdims=True )

    return tf_rad2deg(tf.atan2(y,x))

# takes a 3 dimensional tensor (N, K, 4) and outputs (N,K) angles
def dihedral_tf3(p):
    p0 = tf.gather(p, 0, axis=2)
    p1 = tf.gather(p, 1, axis=2)
    p2 = tf.gather(p, 2, axis=2)
    p3 = tf.gather(p, 3, axis=2)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1, axis=2, keepdims=True))
    b1 = tf.where(tf.is_nan(b1), tf.ones_like(b1), b1)
    
    v = tf.subtract(b0, tf.einsum('bi,bij->bij', tf.einsum('bij,bij->bi', b0, b1), b1))
    w = tf.subtract(b2, tf.einsum('bi,bij->bij', tf.einsum('bij,bij->bi', b2, b1), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 2, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 2, keepdims=True )

    return tf.atan2(y,x)

In [30]:
p0 = np.array([24.969, 13.428, 30.692]) # N
p1 = np.array([24.044, 12.661, 29.808]) # CA
p2 = np.array([22.785, 13.482, 29.543]) # C
p3 = np.array([21.951, 13.670, 30.431]) # O
# result: -71

p_org = np.array([[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ],[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ],[
                p0,
                p1,
                p2,
                p3,
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ]])
print(p_org.shape)

p = tf.convert_to_tensor(p_org[2])
# p1 = np.expand_dims(p1, axis=0)
p = p[None,:,:,None]
p = tf.extract_image_patches(p,
  ksizes=[1, 4, 3, 1],
  strides=[1, 1, 1, 1],
  rates=[1, 1, 1, 1],
  padding='VALID')
p = tf.reshape(tf.squeeze(p), [-1, 4, 3])
angles = dihedral_tf1(tf.convert_to_tensor(p_org[2, 0:4]))
angles2 = dihedral_tf2(p)

p2 = tf.convert_to_tensor(p_org)
# p1 = np.expand_dims(p1, axis=0)
p2 = p2[:,:,:,None]
p2 = tf.extract_image_patches(p2,
  ksizes=[1, 4, 3, 1],
  strides=[1, 1, 1, 1],
  rates=[1, 1, 1, 1],
  padding='VALID')
p2 = tf.reshape(tf.squeeze(p2), [3, -1, 4, 3])
angles3 = dihedral_tf3(p2)

with tf.Session() as sess:
    angles_, angles2_, angles3_, pshape, p2shape = sess.run([angles, angles2, angles3, tf.shape(p), tf.shape(p2)])
    
np.array(angles_), np.array(angles2_), np.array(angles3_[2])

(3, 7, 3)


(array(-71.21515115), array([[ -71.21515115],
        [-127.82961937],
        [   0.        ],
        [          nan]]), array([[-1.24293886],
        [-2.23104774],
        [ 0.        ],
        [ 0.        ]]))

In [44]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PPBuilder
parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure("2gb1", "2gb1.pdb")
ppb=PPBuilder()
protein = []
for pp in ppb.build_peptides(structure):
    for i,res in enumerate(pp):
        protein.append([res["N"].get_coord(), res["CA"].get_coord(), res["C"].get_coord()])
    diheadrals = np.array(pp.get_phi_psi_list())

In [47]:
protein = np.array(protein).reshape(-1, 3)
protein = np.expand_dims(protein, axis=0)

In [48]:
protein.shape, diheadrals.shape

((1, 168, 3), (56, 2))

In [144]:
p2 = tf.convert_to_tensor(np.concatenate([protein, protein]))
print(p2.get_shape())
# p2 = np.expand_dims(p2, axis=0)
p2 = p2[:,:,:,None]
p2 = tf.extract_image_patches(p2,
  ksizes=[1, 4, 3, 1],
  strides=[1, 1, 1, 1],
  rates=[1, 1, 1, 1],
  padding='VALID')
p2 = tf.reshape(tf.squeeze(p2), [2, -1, 4, 3])
angles3 = dihedral_tf3(p2)

with tf.Session() as sess:
    angles3_, pshape, p2shape = sess.run([angles3, tf.shape(p), tf.shape(p2)])

(2, 168, 3)


In [153]:
angles3_ = np.insert(angles3_, 0, None, axis=1)

In [172]:
psi_and_phi_i = np.array(np.sort(list(range(0,angles3_.shape[1],3)) + list(range(1,angles3_.shape[1],3))))
psi_and_phi_i

array([  0,   1,   3,   4,   6,   7,   9,  10,  12,  13,  15,  16,  18,
        19,  21,  22,  24,  25,  27,  28,  30,  31,  33,  34,  36,  37,
        39,  40,  42,  43,  45,  46,  48,  49,  51,  52,  54,  55,  57,
        58,  60,  61,  63,  64,  66,  67,  69,  70,  72,  73,  75,  76,
        78,  79,  81,  82,  84,  85,  87,  88,  90,  91,  93,  94,  96,
        97,  99, 100, 102, 103, 105, 106, 108, 109, 111, 112, 114, 115,
       117, 118, 120, 121, 123, 124, 126, 127, 129, 130, 132, 133, 135,
       136, 138, 139, 141, 142, 144, 145, 147, 148, 150, 151, 153, 154,
       156, 157, 159, 160, 162, 163, 165])

In [173]:
phi_and_psi

array([nan, 2.0956292152404785, -2.2517926692962646, 2.058528423309326,
       -1.6022758483886719, 2.867117166519165, -1.9393948316574097,
       2.0755319595336914, -1.5184667110443115, 2.557396411895752,
       -2.1848230361938477, 1.8136522769927979, -1.3834537267684937,
       2.005882978439331, -1.84293794631958, 0.6435086131095886,
       -0.9751341342926025, 2.513482093811035, -1.0469741821289062,
       -1.022780179977417, -1.5940858125686646, 0.14123353362083435,
       -2.7431325912475586, 2.390169143676758, -2.3366808891296387,
       2.5530786514282227, -2.931011199951172, -2.8704309463500977,
       -3.0626862049102783, 2.284626007080078, -2.585129737854004,
       2.6722536087036133, -1.8480744361877441, 2.4088892936706543,
       -2.1858036518096924, 2.6633639335632324, -1.642419457435608,
       2.1030330657958984, -2.427008867263794, 2.559390068054199,
       -1.4653993844985962, -0.38810446858406067, -2.55434250831604,
       3.030111074447632, -1.255328893661499, -0

In [174]:
phi_and_psi = angles3_[0][psi_and_phi_i][:,0]
phi_and_psi = np.append(phi_and_psi, None)
angles = phi_and_psi.reshape(-1,2)
# angles3_[0][:10]

In [183]:
angles = np.asarray(angles, dtype=np.float32)
diheadrals = np.asarray(diheadrals, dtype=np.float32)

np.mean(np.isclose(angles, diheadrals, equal_nan=True)), np.mean(np.isclose(np.zeros_like(diheadrals), diheadrals, equal_nan=True))

(1.0, 0.0)